## Task 1. Decorators
1. Write a decorator, which checks complience of function's signature to given interface.
2. Write a decorator, which decorates function with a provided decorator.
3. Write a decorator, which adds benchmarking information (function execution time) to function.
4. 
Write a decorator, which logs function's calls (prints function names and values of functions arguments.)

### Task 2. Configurable pipeline of integer data transformations
Assume we have a list of integers, which represents some abstract collected data. Create a class for creating configurable pipelines of transformations for the provided list of integers. The following transformations should be supported:
1. Convertion to ratios (every element is divided by the sum of all list elements)
2. Removal of the list mean value
3. Filtering of negative or zero elements

## Task 3. Text preprocessing
Assume we have an arbitrary text and plan to prepare it for a further data analysis. The text might contain html tags, emails, latex commands. The task is to create a configurable text preprocessor, which allows to remove certain elements from provided text. The following filters should be supported:
1. Removal of punctuation symbols from a given list 
2. Removal of stop words from a given list
3. Removal of HTML tags
4. Removal of e-mails